In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
import torch
torch.cuda.is_available()

True

In [5]:

!pip install git+https://github.com/afnan47/cuda.git



  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-vohuw3r2
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-vohuw3r2
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4290 sha256=b57234f492db588cac1c302f5ec40f8b644d5ec8f6c6c8a8c35f8424366ed2bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-pygr5cps/wheels/bc/4e/e0/2d86bd15f671dbeb32144013f1159dba09757fde36dc51a963
Successfully built NVCCPlugin


In [11]:
# Step 2: Write the CUDA code to a file
cuda_code = r"""
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdlib.h>

#define N 500

__global__ void add(int *a, int *b, int *c) {
    int tid = threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main(void) {
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;

    cudaError_t err = cudaSuccess;

    err = cudaMalloc((void**)&dev_a, N * sizeof(int));
    if (err != cudaSuccess) {
        printf("Failed to allocate on device\n");
        printf("Error is: %s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    cudaMalloc((void**)&dev_b, N * sizeof(int));
    cudaMalloc((void**)&dev_c, N * sizeof(int));

    for (int i = 0; i < N; i++) {
        a[i] = i;
        b[i] = i * i;
        c[i] = 0;
    }

    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    cudaMemcpy(dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_c, c, N * sizeof(int), cudaMemcpyHostToDevice);

    add<<<1, N>>>(dev_a, dev_b, dev_c);

    err = cudaMemcpy(c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("Failed to copy from device\n");
        exit(EXIT_FAILURE);
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, start, end);
    printf("Name: Chinmayee Bondgulwar BACO21157\n");
    printf(" ");

    printf("Execution time = %f ms\n", time);

    // Print a few sample results
    for (int i = 0; i < 10; i++) {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}
"""

with open('add_vectors.cu', 'w') as f:
    f.write(cuda_code)



In [12]:
# Step 3: Compile and run
!nvcc -o add_vectors add_vectors.cu


In [13]:
!./add_vectors

Name: Chinmayee Bondgulwar BACO21157
 Execution time = 8.147104 ms
0 + 0 = 0
1 + 1 = 0
2 + 4 = 0
3 + 9 = 0
4 + 16 = 0
5 + 25 = 0
6 + 36 = 0
7 + 49 = 0
8 + 64 = 0
9 + 81 = 0
